In [ ]:
import sys
sys.path.insert(0, '../toys-data')
import conf

In [ ]:
import requests
import time
import random
import datetime

from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import ssl
import smtplib

In [ ]:
project_id = '2024030600'

In [ ]:
giving_message = '한글테스트'

In [ ]:
def o(*args):
    message = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') + '\t'
    for m in args:
        message += str(m)
        if m != args[-1]:
            message += '\t'
    print(message, file=sys.stderr)

In [ ]:
def mail():
    # SMTP
    SMTP_SERVER = conf.SMTP_SERVER
    SMTP_PORT = conf.SMTP_PORT
    SMTP_USER = conf.SMTP_USER
    SMTP_PASSWORD = conf.SMTP_PASSWORD
    
    # mime
    msg = MIMEMultipart()
    msg['From'] = conf.SMTP_FROM
    msg['To'] = conf.SMTP_TO
    msg['Subject'] = giving_message

    mimetext = MIMEText(giving_message, _charset='utf-8')
    msg.attach(mimetext)

    # smtp
    # ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
    ssl_context = ssl.create_default_context()
    ssl_context.set_ciphers('DEFAULT:!DH')
    smtp = smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT, context=ssl_context)
    smtp.login(SMTP_USER, SMTP_PASSWORD)
    smtp.sendmail(msg['From'], msg['To'], msg.as_string())
    smtp.close()

    o('[INFO] smtp done')

def slack():
    requests.post(
    f'https://hooks.slack.com/services/{conf.SLACK_WEBHOOK_URL}',
    headers={
        'content-type': 'application/json'
    },
    json={
        'text': giving_message,
        'blocks': [
        {
            'type': 'section',
            'text': {
            'type': 'mrkdwn',
            'text': giving_message
            }
        }
        ]
    }
    )
    o('[INFO] slack done')

In [ ]:
def try_get_response():
    try:
        headers = conf.get_request_header(project_id)
        base_url = conf.get_base_url(project_id)
        return requests.get(base_url, headers=headers)
    except Exception as e:
        o('[EXCEPT] try_get_response')
        o('', e)
        return None
def try_get_json(response):
    try:
        return response.json()
    except Exception as e:
        o('[EXCEPT] try_get_json')
        o('', e)
        o('', response.text)
        return None
def try_get_remain_seat(json_data):
    try:
        return int(json_data['data']['remainSeat'][1]['remainCnt'])
    except Exception as e:
        o('[EXCEPT] try_get_remain_seat')
        o('', e)
        o('', json_data)
        return None
def try_alarm():
    try:
        mail()
    except Exception as e:
        o('[EXCEPT] try_smtp')
        o('', e)
    try:
        slack()
    except Exception as e:
        o('[EXCEPT] try_slack')
        o('', e)
    return
    import winsound
    import time
    frequency = 1500  # Set Frequency To 2500 Hertz
    duration = 1000  # Set Duration To 1000 ms == 1 second
    for _ in range(20):
        time.sleep(0.5)
        winsound.Beep(frequency, duration)        

In [ ]:
def check():
    response = try_get_response()
    if response == None:
        return False
    json_data = try_get_json(response)

    if json_data == None:
        return False

    remain_seat = try_get_remain_seat(json_data)
    if remain_seat == None:
        return False
    o('남은자리', f'{remain_seat}')
    if remain_seat > 0:
        try_alarm()
    return True


In [ ]:
# try_alarm()
# check()
# slack()

In [ ]:
# while True:
#     if check() == False:
#         time.sleep(random.randrange(50 * 2, 70 * 2))
#     else:
#         time.sleep(random.randrange(50, 70))